In [1]:
from lib.logging import logger

In [2]:
from pipeline import SettlementPipeline
from transaction_parser import StringTransactionInputParser

allTransactions = '''
group A B C D E F
A 40 D
A 10 E
B 30 F
B 30 E
C 70 E
'''

pipeline = SettlementPipeline(inputParserKlass=StringTransactionInputParser, nParticipantsThreshold=8)
pipeline.getSettlementTransactions(allTransactions)

2026-02-08 03:59:34.72India Standard Time 19224 10372 INFO Settling with config: Settler(nParticipantsThreshold=8)
2026-02-08 03:59:34.73India Standard Time 19224 10372 INFO Using MinimumTransactionSettler settlement strategy for 6 participants
2026-02-08 03:59:34.73India Standard Time 19224 10372 INFO Number of transactions: 4
2026-02-08 03:59:34.74India Standard Time 19224 10372 INFO D has to pay C 40.0
2026-02-08 03:59:34.74India Standard Time 19224 10372 INFO E has to pay B 60.0
2026-02-08 03:59:34.74India Standard Time 19224 10372 INFO E has to pay A 50.0
2026-02-08 03:59:34.75India Standard Time 19224 10372 INFO F has to pay C 30.0


In [3]:
pipeline = SettlementPipeline(inputParserKlass=StringTransactionInputParser, nParticipantsThreshold=5)
pipeline.getSettlementTransactions(allTransactions)

2026-02-08 03:59:34.89India Standard Time 19224 10372 INFO Settling with config: Settler(nParticipantsThreshold=5)
2026-02-08 03:59:34.90India Standard Time 19224 10372 INFO Using GreedyTransactionSettler settlement strategy for 6 participants
2026-02-08 03:59:34.90India Standard Time 19224 10372 INFO Number of transactions: 5
2026-02-08 03:59:34.91India Standard Time 19224 10372 INFO D has to pay B 20.0
2026-02-08 03:59:34.92India Standard Time 19224 10372 INFO D has to pay A 20.0
2026-02-08 03:59:34.93India Standard Time 19224 10372 INFO E has to pay C 70.0
2026-02-08 03:59:34.94India Standard Time 19224 10372 INFO E has to pay B 40.0
2026-02-08 03:59:34.95India Standard Time 19224 10372 INFO F has to pay A 30.0


In [4]:
allTransactions = '''
group A B C D E F
A 40 D
A 10 E
B 30 F
B 30 E,
C 70 E
'''
try:
    pipeline.getSettlementTransactions(allTransactions)
except ValueError as ve:
    logger.exception(f'value error: {ve}')

2026-02-08 03:59:34.104India Standard Time 19224 10372 ERROR value error: invalid chars present in the string
Traceback (most recent call last):
  File "C:\Users\suraj\AppData\Local\Temp\ipykernel_19224\304248226.py", line 10, in <module>
    pipeline.getSettlementTransactions(allTransactions)
  File "c:\Users\suraj\projects\cospend\pipeline.py", line 22, in getSettlementTransactions
    self._validate(inputTransactions)
  File "c:\Users\suraj\projects\cospend\pipeline.py", line 15, in _validate
    self.inputParserKlass.validate(inputString)
  File "c:\Users\suraj\projects\cospend\transaction_parser.py", line 42, in validate
    cls.validateChars(inputString)
  File "c:\Users\suraj\projects\cospend\transaction_parser.py", line 37, in validateChars
    raise ValueError('invalid chars present in the string')
ValueError: invalid chars present in the string


In [ ]:
import datetime

class SpendingRecord:
    def __init__(self, amount:float|int, spender:str, members:tuple|None = None, note:str|None = ''):
            self.entryTime = datetime.datetime.now()
            self.amount = amount
            self.spender = spender
            self.members = members or []
            self.note = note

    def __repr__(self):
        return f'created @ {self.entryTime} {self.__class__.__name__}(amount={self.amount}, spender={self.spender}, members={self.members}, note={self.note or ""})'

    def logRecord(self):
        return f'{self.spender} {self.amount} {' '.join(self.members)}'
    
    def __str__(self):
        res = f'{self.entryTime} {self.spender} spent {self.amount}'
        if self.members:
            res = f'{res} on {self.members}'
        if self.note:
            res = f'{res} for {self.note}'
        return res

class SpendingTransactionRecorder:

    BASE_DIR = '/transcations/'

    def __init__(self, groupName:str, members:set):
        self.groupName = groupName
        self.members = members
        self._path = f'{self.BASE_DIR}{self.groupName}.txt'
        self._transactions = []

    def addRecord(self, amount:float|int, spender:str, members:str|set|None = None, note:str|None = ''):
        if isinstance(members, str):
            if members.startswith('!'):
                members = set(m for m in self.members if m!=members[1:])
        members = set(members)
        if members.difference(self.members):
            raise ValueError(f'Unknown member(s) present in the entry: {members.difference(self.members)}')
        self._transactions.append(SpendingRecord(amount, spender, members, note))

    def showRecords(self):
        for entry in self._transactions:
            logger.info(entry)

    def logRecords(self):
        for entry in self._transactions:
            logger.info(entry.logRecord())


In [67]:
import time
import random
randomsleep = lambda: time.sleep(random.randint(1, 2))
recorder = SpendingTransactionRecorder(members=set(['A', 'B', 'C', 'D', 'E', 'F']))
recorder.showRecords()
recorder.addRecord(40, 'A', ['D'], 'coffee')
randomsleep()
recorder.addRecord(10, 'A', ['E'], 'water')
#randomsleep()
recorder.addRecord(30, 'B', ['F'], 'tea')
#randomsleep()
recorder.addRecord(30, 'B', ['E'])
#randomsleep()
recorder.addRecord(70, 'C', '!F', 'snack')
randomsleep()
recorder.showRecords()

2026-02-08 04:27:06.981India Standard Time 19224 10372 INFO 2026-02-08 04:27:04.980083 A spent 40 on {'D'} for coffee
2026-02-08 04:27:06.981India Standard Time 19224 10372 INFO 2026-02-08 04:27:05.980533 A spent 10 on {'E'} for water
2026-02-08 04:27:06.983India Standard Time 19224 10372 INFO 2026-02-08 04:27:05.980533 B spent 30 on {'F'} for tea
2026-02-08 04:27:06.984India Standard Time 19224 10372 INFO 2026-02-08 04:27:05.980533 B spent 30 on {'E'}
2026-02-08 04:27:06.984India Standard Time 19224 10372 INFO 2026-02-08 04:27:05.980533 C spent 70 on {'E', 'D', 'C', 'B', 'A'} for snack


In [68]:
recorder.logRecords()

2026-02-08 04:27:06.994India Standard Time 19224 10372 INFO A 40 D
2026-02-08 04:27:06.995India Standard Time 19224 10372 INFO A 10 E
2026-02-08 04:27:06.995India Standard Time 19224 10372 INFO B 30 F
2026-02-08 04:27:06.996India Standard Time 19224 10372 INFO B 30 E
2026-02-08 04:27:06.997India Standard Time 19224 10372 INFO C 70 E D C B A


In [69]:
for t in recorder._transactions:
    print(repr(t))

created @ 2026-02-08 04:27:04.980083 SpendingRecord(amount=40, spender=A, members={'D'}, note=coffee)
created @ 2026-02-08 04:27:05.980533 SpendingRecord(amount=10, spender=A, members={'E'}, note=water)
created @ 2026-02-08 04:27:05.980533 SpendingRecord(amount=30, spender=B, members={'F'}, note=tea)
created @ 2026-02-08 04:27:05.980533 SpendingRecord(amount=30, spender=B, members={'E'}, note= )
created @ 2026-02-08 04:27:05.980533 SpendingRecord(amount=70, spender=C, members={'E', 'D', 'C', 'B', 'A'}, note=snack)
